In [238]:
import os
import math

import numpy as np
import cv2 as cv
import tensorflow as tf
import tensorflow_datasets as tfds
import random as rn

from keras.models import Model, Sequential
from keras.layers import *
import keras
from keras.applications import mobilenet_v2
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

In [239]:
SEED = 12345
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
rn.seed(SEED)

In [240]:
COLOR_MODE = "G"

EPOCHS = 50
BATCH_SIZE = 512

In [241]:
def LeNet_5():
    model = Sequential()
    model.add(Conv2D(32,(5,5),strides=(1,1),input_shape=(32,32,1),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64,(5,5),strides=(1,1),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,activation='softmax'))
    return model

In [242]:
# Load MobileNet model
model = LeNet_5()
model.summary()
# opt = optimizers.SGD(lr=1e-2, momentum=0.9),
opt = optimizers.Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_41 (Conv2D)           (None, 32, 32, 32)        832       
_________________________________________________________________
batch_normalization_20 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 16, 16, 64)        51264     
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 4096)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 128)             

In [243]:
# Fetch the dataset directly
dataset = tfds.image_classification.Cifar10()

In [244]:
# Download the data, prepare it, and write it to disk
dataset.download_and_prepare()

In [245]:
# Describe the dataset with DatasetInfo
C = dataset.info.features['label'].num_classes
Ntrain = dataset.info.splits['train'].num_examples
Ntest = dataset.info.splits['test'].num_examples
Nbatch = BATCH_SIZE
assert C == 10
assert Ntrain == 50000
assert Ntest == 10000

In [246]:
# Load data from disk as tf.data.Datasets
datasets = dataset.as_dataset()
train_dataset, validation_dataset = datasets['train'], datasets['test']
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(validation_dataset, tf.data.Dataset)

In [247]:
def dataset_generator(dataset, batch_size=256, num_classes=10, is_training=False, color_mode="grayscale"):
  images = np.zeros((batch_size, 32, 32, 1))
  print("intote")
  labels = np.zeros((batch_size, num_classes))
  while True:
    count = 0 
    for sample in tfds.as_numpy(dataset):
      image = sample["image"]
      label = sample["label"]

      if color_mode == "grayscale":
        # rgb to grayscale
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
      else: # keep only one channel in RGB
        channel_index = list('BGR').index(color_mode.upper())
        image = image[:,:,channel_index]

      images[count%batch_size] = mobilenet_v2.preprocess_input(np.expand_dims(cv.resize(image, (32, 32)), -1))
      labels[count%batch_size] = np.expand_dims(to_categorical(label, num_classes=num_classes), 0)
      
      count += 1
      if (count%batch_size == 0):
        yield images, labels

In [248]:
labels = np.zeros((Ntest))
pred_labels = np.zeros((Ntest, C))
pred_labels_new = np.zeros((Ntest, C))

# Train on Cifar10
checkpoint_path = "LeNet_5/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.makedirs(checkpoint_dir, exist_ok=True)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    verbose=1,
    monitor="val_loss",
    mode='auto',
    save_weights_only=True,
    save_best_only=True,
    save_freq='epoch'
)
    
csv_logger = keras.callbacks.CSVLogger('LeNet_5.csv')


model.fit(
    dataset_generator(train_dataset, batch_size=Nbatch, is_training=True, color_mode=COLOR_MODE),
    steps_per_epoch= Ntrain // Nbatch,
    epochs = EPOCHS,
    validation_data = dataset_generator(validation_dataset, batch_size=Nbatch, color_mode=COLOR_MODE),
    validation_steps = Ntest // Nbatch,
    callbacks = [cp_callback, csv_logger]
)

intote
Epoch 1/50
26/97 [=======>......................] - ETA: 21s - loss: 2.5943 - accuracy: 0.1512

KeyboardInterrupt: ignored

In [ ]:

score = model.evaluate_generator(
    dataset_generator(validation_dataset,batch_size=Nbatch),
    color_mode=COLOR_MODE,
    steps= Ntest // Nbatch,
    verbose=1,)

print("Evaluation Result of new Model on cifar100: " + str(score))

In [ ]:
model.save("MobileNetV2.h5")